In [1]:
import pandas as pd

In [2]:
d_xl = pd.read_excel("Lab8_data.xlsx")
d_xl

,age,income,student,credit_rating,buys_computer
0,<=30,high,no,fair,no
1,<=30,high,no,excellent,no
2,31…40,high,no,fair,yes
3,>40,medium,no,fair,yes
4,>41,low,yes,fair,yes
5,>42,low,yes,excellent,no
6,31…40,low,yes,excellent,yes
7,<=30,medium,no,fair,no
8,<=30,low,yes,fair,yes
9,>40,medium,yes,fair,yes


# A1

In [3]:
print("Prior probabilties for each features w.r.t 'buys_computer'\n")
for column in d_xl.columns:
    if column != 'buys_computer':  
        print(f"\nPrior Probabilities for {column.upper()}:")
        prior_probabilities = d_xl.groupby(column)['buys_computer'].value_counts(normalize=True)
        print(prior_probabilities)
        print()

Prior probabilties for each features w.r.t 'buys_computer'


Prior Probabilities for AGE:
age    buys_computer
31…40  yes              1.000000
<=30   no               0.600000
       yes              0.400000
>40    yes              0.666667
       no               0.333333
>41    yes              1.000000
>42    no               1.000000
Name: proportion, dtype: float64


Prior Probabilities for INCOME:
income  buys_computer
high    no               0.500000
        yes              0.500000
low     yes              0.750000
        no               0.250000
medium  yes              0.666667
        no               0.333333
Name: proportion, dtype: float64


Prior Probabilities for STUDENT:
student  buys_computer
no       no               0.571429
         yes              0.428571
yes      yes              0.857143
         no               0.142857
Name: proportion, dtype: float64


Prior Probabilities for CREDIT_RATING:
credit_rating  buys_computer
excellent      no              

In [4]:
print("Prior probabilities for the 'buys_computer' classes\n")
total_instances = len(d_xl)
buys_computer_counts = d_xl['buys_computer'].value_counts()

prior_prob_yes = buys_computer_counts['yes'] / total_instances
prior_prob_no = buys_computer_counts['no'] / total_instances

print(f"Prior probability for buys_computer = 'yes': {prior_prob_yes}")
print(f"Prior probability for buys_computer = 'no': {prior_prob_no}")

Prior probabilities for the 'buys_computer' classes

Prior probability for buys_computer = 'yes': 0.6428571428571429
Prior probability for buys_computer = 'no': 0.35714285714285715


# A2

In [5]:
from sklearn.neighbors import KernelDensity
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import LabelEncoder as LE

label_encoder = LE()
dxl_encoded = d_xl.copy()
for col in d_xl.columns:
    if d_xl[col].dtype == 'object':
        dxl_encoded[col] = label_encoder.fit_transform(d_xl[col])


X = dxl_encoded.loc[:, 'age':'credit_rating']
y = dxl_encoded['buys_computer']  

X_train, X_test, y_train, y_test = tts(X, y, test_size=0.2, random_state=42)

model = GaussianNB()
model.fit(X_train, y_train)

for col in X.columns:
    unique_values = X[col].unique()
    print()
    for value in unique_values:
        for class_val in model.classes_:
            subset_data = X[(X[col] == value) & (y == class_val)]
            
            # If subset_data is empty, continue to the next iteration
            if len(subset_data) == 0:
                print(f"Class-conditional density for {col} = {value} and class {class_val}: 0 (No data)")
                continue
            
            # Calculate KDE for the subset data
            kde = KernelDensity()
            kde.fit(subset_data[[col]])
            density = len(subset_data) / len(X) * sum(kde.score_samples(subset_data[[col]]))
            
            print(f"Class-conditional density for {col} = {value} and class {class_val}: {density}")


Class-conditional density for age = 1 and class 0: -0.5907461999172895
Class-conditional density for age = 1 and class 1: -0.2625538666299065
Class-conditional density for age = 0 and class 0: 0 (No data)
Class-conditional density for age = 0 and class 1: -1.050215466519626
Class-conditional density for age = 2 and class 0: -0.06563846665747662
Class-conditional density for age = 2 and class 1: -0.2625538666299065
Class-conditional density for age = 3 and class 0: 0 (No data)
Class-conditional density for age = 3 and class 1: -0.06563846665747662
Class-conditional density for age = 4 and class 0: -0.06563846665747662
Class-conditional density for age = 4 and class 1: 0 (No data)

Class-conditional density for income = 0 and class 0: -0.2625538666299065
Class-conditional density for income = 0 and class 1: -0.2625538666299065
Class-conditional density for income = 2 and class 0: -0.2625538666299065
Class-conditional density for income = 2 and class 1: -1.050215466519626
Class-condition

# A3

In [6]:
from scipy.stats import chi2_contingency as cc

res = []

for col1 in X.columns:
    for col2 in X.columns:
        if col1 != col2:
            contingency_table = pd.crosstab(X[col1], X[col2])
            chi2, p, _, _ = cc(contingency_table)
            
            independence = 'Independent' if p > 0.05 else 'Dependent'
            
            res.append({
                'Feature_1': col1,
                'Feature_2': col2,
                'Chi2_Statistic': chi2,
                'P_Value': p,
                'Independence': independence
            })

res_df = pd.DataFrame(res) 
print(res_df)

        Feature_1      Feature_2  Chi2_Statistic   P_Value Independence
0             age         income       10.325000  0.242952  Independent
1             age        student        2.533333  0.638677  Independent
2             age  credit_rating        2.294444  0.681781  Independent
3          income            age       10.325000  0.242952  Independent
4          income        student        5.666667  0.058816  Independent
5          income  credit_rating        0.729167  0.694486  Independent
6         student            age        2.533333  0.638677  Independent
7         student         income        5.666667  0.058816  Independent
8         student  credit_rating        0.000000  1.000000  Independent
9   credit_rating            age        2.294444  0.681781  Independent
10  credit_rating         income        0.729167  0.694486  Independent
11  credit_rating        student        0.000000  1.000000  Independent


# A4

In [7]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import LabelEncoder as LE

label_encoder = LE()
dxl_encoded = d_xl.copy()
for col in d_xl.columns:
    if d_xl[col].dtype == 'object':
        dxl_encoded[col] = label_encoder.fit_transform(d_xl[col])


X = dxl_encoded.loc[:, 'age':'credit_rating']
y = dxl_encoded['buys_computer']  

X_train, X_test, y_train, y_test = tts(X, y, test_size=0.2, random_state=42)

model = GaussianNB()
model.fit(X_train, y_train)

predicted = model.predict(X_test)

print(X_test)

print("\nPredicted classes for test data:")
print(predicted)

    age  income  student  credit_rating
9     2       2        1              1
11    0       2        0              0
0     1       0        0              1

Predicted classes for test data:
[1 0 1]


# A5

In [8]:
data = pd.read_csv("NID_data.csv")
data

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25187,0,tcp,exec,RSTO,0,0,0,0,0,0,...,7,0.03,0.06,0.00,0.00,0.00,0.00,1.00,1.00,anomaly
25188,0,tcp,ftp_data,SF,334,0,0,0,0,0,...,39,1.00,0.00,1.00,0.18,0.00,0.00,0.00,0.00,anomaly
25189,0,tcp,private,REJ,0,0,0,0,0,0,...,13,0.05,0.07,0.00,0.00,0.00,0.00,1.00,1.00,anomaly
25190,0,tcp,nnsp,S0,0,0,0,0,0,0,...,20,0.08,0.06,0.00,0.00,1.00,1.00,0.00,0.00,anomaly


In [9]:
from sklearn.model_selection import train_test_split as tts
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder as LE

data = data.dropna()
le_protocol = LE()
le_service = LE()
le_flag = LE()
le_class = LE()

data['protocol_type'] = le_protocol.fit_transform(data['protocol_type'])
data['service'] = le_service.fit_transform(data['service'])
data['flag'] = le_flag.fit_transform(data['flag'])
data['class'] = le_class.fit_transform(data['class'])

X = data.drop('class', axis=1)
y = data['class']

X_train, X_test, y_train, y_test = tts(X, y, test_size=0.2, random_state=42)

nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)
y_pred = nb_classifier.predict(X_test)

accuracy = metrics.accuracy_score(y_test, y_pred)
precision = metrics.precision_score(y_test, y_pred, average='weighted')
recall = metrics.recall_score(y_test, y_pred, average='weighted')
f1_score = metrics.f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)
 

Accuracy: 0.5572534232982734
Precision: 0.6647941945496836
Recall: 0.5572534232982734
F1 Score: 0.43747403728594875
